In [ ]:
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(9)

sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 8))

In [ ]:

df = pl.read_csv(
    "dataset.csv",
    null_values=["", " ", "NA", "N/A", "nan", "NaN", "null", "Null"],
)
df.head()

In [ ]:
df.describe()

In [ ]:
pl.DataFrame(
    {
        "column": df.columns,
        "null_count": df.null_count().row(0),
        "total_rows": df.height,
    }
).with_columns(
    [
        (pl.col("null_count") / pl.col("total_rows") * 100)
        .round(2)
        .alias("perc %")
    ]
).filter(pl.col("null_count") > 0).sort(
    "null_count", descending=True
)

In [ ]:
# df = df.drop("Fugitive Emissions (Mt)")
threshold = 0.5
columns_to_drop = [
    col for col, null_count in zip(df.columns, df.null_count().row(0))
    if null_count / df.height > threshold
]
df = df.drop(columns_to_drop)
df.columns
print(f"Columns to be dropped: {columns_to_drop}")

In [ ]:
new = df.with_columns([
   pl.col(col).fill_null(pl.col(col).mean()) for col in df.select(cs.numeric()).columns
]).head()

In [ ]:
numerical_cols = df.select(cs.numeric()).columns
numerical_cols